In [2]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import pyproj
import folium
import folium.features as ftr
from folium.features import DivIcon
import shapely
import shapely.wkt
import shapely.ops as ops
from shapely.ops import transform

In [3]:
# connection to database:
try:
    conn = psycopg2.connect("dbname='spatial' user='postgres' host='localhost' password='deeznuts'")
except:
    print("cant connect to the database")

In [4]:
sql1 = '''select parid, luc, luc_desc, aprland, total_area, ST_AsText(geom) as wkt, price from volusia.sales_analysis 
	where luc = '0100' and 
	rmbed = 3 limit 100'''
df  = pd.read_sql_query(sql1, conn)
df = df.dropna()

In [5]:
df.head()

,parid,luc,luc_desc,aprland,total_area,wkt,price
0,3418719.0,0100,Single Family,55897.0,2262.0,MULTIPOLYGON(((659266.8390000015 1764621.15625...,291000.0
1,3418751.0,0100,Single Family,51597.0,2466.0,MULTIPOLYGON(((659021.9420000017 1764621.71199...,235000.0
2,3419588.0,0100,Single Family,51975.0,2548.0,MULTIPOLYGON(((658580.6000000015 1766075.05799...,272400.0
3,3419782.0,0100,Single Family,51975.0,2412.0,MULTIPOLYGON(((659012.9842500016 1765825.77199...,185500.0
4,3420276.0,0100,Single Family,83491.0,2143.0,MULTIPOLYGON(((657562.1110000014 1766465.35000...,224000.0


In [6]:
wgs84 = pyproj.CRS('EPSG:4326')
fl_sp_east_feet = pyproj.CRS('EPSG:2236')
project = pyproj.Transformer.from_crs(fl_sp_east_feet, wgs84, always_xy=True).transform 

In [7]:
# start our map over ERAU.
m = folium.Map(location=[29.1887876219045, -81.0494807582431], zoom_start=18, width="100%")

def highlight_function(feature):  # how to color the features when highlighted
    return{
        'weight': 5,
        'color': '#626',
        'dashArray': '',
        'fillOpacity': 0.7
    };

def style_function(feature):     # how to color the features we are adding to the map
    return{
        'weight': 1.5,
        #'color': 'blue',
        'dashArray': '5, 5',
        'fillOpacity': 0.7
    };

fgv = folium.FeatureGroup(name="parcels")   # create a folium map layer feature group, then add polygons, then add to map m
fgv2 = folium.FeatureGroup(name="parcel_labels")   # create a folium map layer feature group, then add polygons, then add to map m

for index, row in df.iterrows():    
    tooltip_show = str(int(row.price))      # 
    geom = shapely.wkt.loads(row.wkt)  # use shapely wkt.loads to load WKT into a shapeply geometry (polygon)
    wgs_geom = transform(project, geom)     # reproject that polygon to wgs84
    
    # create geoJSON Features, add a popup to them, then add to the feature group
    c= folium.GeoJson(shapely.geometry.asShape(wgs_geom),  # convert the wgs84 shapely geom to JSON and add to group 
                       name='parcels',      # add all these geometries to feature group named parcels?
                       overlay=True,
                       style_function=style_function,  # how to color the feature
                       highlight_function=highlight_function, # how to color/symbolize on highlight/mouseover
                       tooltip=tooltip_show   # on hover over show this information,
                       #zoom_on_click=True     # does not work on my version of folium
    )
    # generate a folium Popup then add it to the GeoJson object c, format all data into a table
    popup_string = '<table>'
    for i, v in row.iteritems():
        popup_string += "<tr><td>%10s</td><td>&nbsp;" % i.title() + " " + str(v) + "</td></tr>\n"
    popup_string += "</table>"
    folium.Popup(popup_string).add_to(c)    # add this popup to the c object
    
    # now that we've created a GeoJson folium feature, with popups, add it to the feature group
    fgv.add_child(c)
    
    # We also want to add a label for each parcel on the map. Create a separate folium Markers for each label
    # since we already have the the shapely library loaded, can quickly get access to the geometries centroid
    # grab the lat, lon of the centroid of each parcel and add to map as a Marker
    # https://stackoverflow.com/questions/46400769/numbers-in-map-marker-in-folium
    # have to put at centroid, format text as html div
    divlabel = '<div style="font-size: 10pt; color : black">' + tooltip_show + '</div>'
    marker = folium.Marker(
        [wgs_geom.centroid.y, wgs_geom.centroid.x], 
        icon=DivIcon(html=divlabel)
    ).add_to(fgv2)   # added to second feature group, so that we can control separately

# add created map layers to map
fgv.add_to(m)    # now add the whole feature group to the map, which we call parcels
fgv2.add_to(m)   # add parcel_labels

m.add_child(folium.LayerControl(position='topright', collapsed=False))

m.fit_bounds(fgv.get_bounds())  # set the map bounds to the feature group bounds  wgs84 

In [8]:
#Random sample of houses shows that many 3 bedrooms are near the railroad
#Note that the query did not filter for rrdistance at all. This is all from the sales table
m

In [11]:
# Connecting to AWS and querying sales table
try:
    conn = psycopg2.connect("dbname='spatial' user='postgres' host='pg-db.cq2sbfxn2nv4.us-east-2.rds.amazonaws.com' password='nope.'")
except:
    print("cant connect to the database")

In [12]:
sql2 = 'select * from volusia.sales_analysis'
aws_frame  = pd.read_sql_query(sql2, conn)
aws_frame = aws_frame.dropna()
aws_frame.head()

,parid,luc,luc_desc,aprland,rmbed,total_area,geom,price
0,3418719.0,100,Single Family,55897.0,3.0,2262.0,0106000020BC0800000100000001030000000100000006...,291000.0
1,3418751.0,100,Single Family,51597.0,3.0,2466.0,0106000020BC0800000100000001030000000100000005...,235000.0
2,3419456.0,100,Single Family,74419.0,2.0,1638.0,0106000020BC0800000100000001030000000100000009...,255000.0
3,3419553.0,100,Single Family,61945.0,2.0,1493.0,0106000020BC0800000100000001030000000100000007...,235000.0
4,3419588.0,100,Single Family,51975.0,3.0,2548.0,0106000020BC0800000100000001030000000100000008...,272400.0
